<a href="https://colab.research.google.com/github/arangodb/interactive_tutorials/blob/master/notebooks/ML_Collab_Article/example_output/Arangopipe_View_TF_Visualization_output.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<font color='red'>THIS NOTEBOOK IS FROM THE ARANGOML MULTI-MODEL COLLABORATION ARTICLE. PLEASE REFER TO THAT ARTICLE FOR FURTHER CONTEXT [HERE](https://www.arangodb.com/2021/01/arangoml-series-multi-model-collaboration/).</font>

## Retreiving a Data Visualization (done by a colleague)

### Install pre-requisites

In [1]:
%%capture
!pip install python-arango
!pip install arangopipe==0.0.6.9.3
!pip install pandas PyYAML==5.1.1 sklearn2
!pip install jsonpickle
!pip install tensorflow==2.2.0
# Install TensorFlow
try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x
except Exception:
  pass

print('Installing TensorFlow Data Validation')
!pip install -q tensorflow_data_validation[visualization]

In [2]:
# Must restart runtime for tensorflow_data_validation due to how Colab installs packages.
# You will need to resume running the code blocks below. 
# To resume onced exited, click into the next cell and then CTRL+F10.
exit()

In [1]:
import tensorflow as tf
import tensorflow_data_validation as tfdv
print('TFDV version: {}'.format(tfdv.version.__version__))

TFDV version: 0.26.0


## Connect to Arangopipe


You obtained the credentials from a project administrator or colleague

In [9]:
from arangopipe.arangopipe_storage.arangopipe_api import ArangoPipe
from arangopipe.arangopipe_storage.arangopipe_admin_api import ArangoPipeAdmin
from arangopipe.arangopipe_storage.arangopipe_config import ArangoPipeConfig
from arangopipe.arangopipe_storage.managed_service_conn_parameters import ManagedServiceConnParam
mdb_config = ArangoPipeConfig()
msc = ManagedServiceConnParam()
conn_params = { msc.DB_SERVICE_HOST : "arangoml.arangodb.cloud", \
                        msc.DB_SERVICE_END_POINT : "createDB",\
                        msc.DB_SERVICE_NAME : "createDB",\
                        msc.DB_NAME: 'ML_Collaboration',\
                        msc.DB_USER_NAME:'ML_ReadOnly',\
                        msc.DB_PASSWORD: 'ML_ReadOnly',\
                        msc.DB_SERVICE_PORT : 8529,\
                        msc.DB_CONN_PROTOCOL : 'https',}
mdb_config = mdb_config.create_connection_config(conn_params)
admin = ArangoPipeAdmin(reuse_connection = True, config = mdb_config)
ap_config = admin.get_config()
ap = ArangoPipe(config = ap_config)

# print the login credentials (ML_ReadOnly has read-only permissions)
mdb_config.get_cfg()

Host Connection: https://arangoml.arangodb.cloud:8529


{'arangodb': {'DB_service_host': 'arangoml.arangodb.cloud',
  'DB_end_point': 'createDB',
  'DB_service_name': 'createDB',
  'dbName': 'ML_Collaboration',
  'username': 'ML_ReadOnly',
  'password': 'ML_ReadOnly',
  'DB_service_port': 8529,
  'conn_protocol': 'https'},
 'mlgraph': {'graphname': 'enterprise_ml_graph'}}

## Retrieve the Dataset
Note: Visualization is stored as dataset meta-data

In [3]:
dataset = ap.lookup_dataset("california-housing-dataset")
print(dataset)

{'_key': '545899472', '_id': 'datasets/545899472', '_rev': '_buZsY4i---', 'description': 'data about housing in California', 'encoded_schema': '{\n  "feature": [\n    {\n      "name": "lat",\n      "type": "FLOAT",\n      "presence": {\n        "minFraction": 1.0,\n        "minCount": "1"\n      },\n      "shape": {\n        "dim": [\n          {\n            "size": "1"\n          }\n        ]\n      }\n    },\n    {\n      "name": "long",\n      "type": "FLOAT",\n      "presence": {\n        "minFraction": 1.0,\n        "minCount": "1"\n      },\n      "shape": {\n        "dim": [\n          {\n            "size": "1"\n          }\n        ]\n      }\n    },\n    {\n      "name": "housingMedAge",\n      "type": "INT",\n      "presence": {\n        "minFraction": 1.0,\n        "minCount": "1"\n      },\n      "shape": {\n        "dim": [\n          {\n            "size": "1"\n          }\n        ]\n      }\n    },\n    {\n      "name": "totalRooms",\n      "type": "INT",\n      "pres

## Convert JSON encoded data to TFX artifact

In [4]:
retrieved_stats = dataset["encoded_stats"]
retrieved_schema = dataset["encoded_schema"]

In [5]:
from google.protobuf import json_format
from tensorflow_metadata.proto.v0 import statistics_pb2
from tensorflow_metadata.proto.v0 import schema_pb2

In [6]:
remat_stats = json_format.Parse(retrieved_stats, statistics_pb2.DatasetFeatureStatisticsList())
remat_schema = json_format.Parse(retrieved_schema, schema_pb2.Schema())

In [7]:
tfdv.visualize_statistics(remat_stats)